In [1]:
import os
import time
from cryptography.fernet import Fernet
# Shared file as the storage
shared_file = 'shared.txt'

# Load the key from the file
def load_key():
    return open('secret.key', 'rb').read()
cipher_suite = Fernet(load_key())

In [2]:
def read_message():
    with open(shared_file, 'rb') as file:
        encrypted_message = file.read()
        decrypted_message = cipher_suite.decrypt(encrypted_message)
        return decrypted_message.decode(), len(encrypted_message) * 8  # Return the message and its size in bits

def receiver():
    last_check = time.perf_counter()
    last_modified = os.path.getmtime(shared_file)
    data_rates = []  # List to store data rates for averaging

    while True:
        time.sleep(1)  # Check for changes every second
        current_check = time.perf_counter()
        current_modified = os.path.getmtime(shared_file)
        if current_modified != last_modified:
            new_message, message_size_bits = read_message()
            elapsed_time = current_check - last_check
            data_rate = message_size_bits / elapsed_time  # Calculate data rate in bits per second
            data_rates.append(data_rate)  # Add to list for averaging

            print(f"Receiver detected a change in the shared file.")
            print(f"New message: {new_message}")
            print(f"Data exchange speed: {data_rate} bits per second")

            if new_message == "Finish!":
                break
            last_modified = current_modified
            last_check = current_check  # Update last check time

    # Calculate the average data rate
    average_data_rate = sum(data_rates) / len(data_rates)
    print(f"Average data exchange speed: {average_data_rate} bits per second")


In [3]:
receiver()

Receiver detected a change in the shared file.
New message: Covert message
Data exchange speed: 99.34423368068765 bits per second
Receiver detected a change in the shared file.
New message: Hello!
Data exchange speed: 625.5145345729625 bits per second
Receiver detected a change in the shared file.
New message: Finish!
Data exchange speed: 393.12535789765366 bits per second
Average data exchange speed: 372.66137538376796 bits per second
